In [6]:
import pandas as pd
import numpy as np
import time
from alpha_vantage.timeseries import TimeSeries

# read SP500 components 
df_tickers = pd.read_csv("sp500_components.csv")
df_tickers = df_tickers.rename(columns={"Unnamed: 0": "comp_index"})

# creating a list of all tickers
melted_tickers = pd.melt(df_tickers, id_vars="comp_index", var_name="year", value_name="ticker").dropna()
tickers = melted_tickers["ticker"].unique().tolist()
tickers = tickers[0:10]
# replace YOUR_API_KEY with your actual Alpha Vantage API key
ts = TimeSeries(key='V1PFJOMDHX76TFF8', output_format='pandas')

# download the adjusted close price for each ticker
data_frames = []
for ticker in tickers:
    
    print(tickers.index(ticker), ticker)
    
    try:
        data, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
    
        # rename the column and select only the adjusted close price
        data = data[["5. adjusted close"]].rename(columns={"5. adjusted close":"Adj_Close"})
    
        # create ticker column
        data["ticker"] = ticker
    
        # get log-returns
        data["log_rtn"] = np.log(data["Adj_Close"]).diff().fillna(0)
    
        # sort the index in ascending order
        data = data.sort_index()
    
        # add data to data_frames list 
        data_frames.append(data)

    except:

        #  add failed ticker back to the list
        #tickers.append(ticker)
        
        # wait 1 minute before trying again
        time.sleep(60)

# concatenate the data frames into a single data frame
combined_data = pd.concat(data_frames, axis=0)

adj_close_data = combined_data.loc[:, ["ticker", "Adj_Close"]].reset_index().pivot(index="date", columns="ticker", values="Adj_Close")
log_rtn_data = combined_data.loc[:, ["ticker", "log_rtn"]].reset_index().pivot(index="date", columns="ticker", values="log_rtn")

# print the first and last 5 rows of the data 
print(adj_close_data.head())
print(log_rtn_data.tail())

# save csv file
adj_close_data.to_csv("adj_close.csv")
log_rtn_data.to_csv("log_rtn.csv")


0 AAL
1 AAMRQ
2 AAPL
3 ABI
4 ABS
5 ABT
6 ABX
7 ACKH
8 ACV
9 ADM
ticker      AAL  AAMRQ      AAPL  ABI
date                                 
1999-11-01  NaN    NaN  0.589786  NaN
1999-11-02  NaN    NaN  0.609769  NaN
1999-11-03  NaN    NaN  0.619267  NaN
1999-11-04  NaN    NaN  0.635376  NaN
1999-11-05  NaN    NaN  0.671012  NaN
ticker           AAL  AAMRQ      AAPL  ABI
date                                      
2023-04-19  0.023007    NaN  0.005863  NaN
2023-04-20 -0.005988    NaN  0.009829  NaN
2023-04-21  0.005988    NaN -0.001877  NaN
2023-04-24  0.036701    NaN  0.009480  NaN
2023-04-25  0.000000    NaN  0.000000  NaN
